In [93]:
import pandas as pd

In [94]:
large = True

In [95]:
results = pd.read_csv('./results_large.csv' if large else './results.csv', names=['name', 'repo', 'acc', '#w', 'samples', 'time', 'model', 'model_address', 'split'])
results.tail(1)

,name,repo,acc,#w,samples,time,model,model_address,split
159,normal,all,0.41,52,3529,31-05-2022_04-22-41,t5-large,/scratch/arminz/tmp/currl_/data/all/data/svgdo...,source-test


In [96]:
results = results[results['samples'] > 33]

In [97]:
results = results[results['model_address'].apply(lambda m: not ('svg.js' in m or 'foam' in m))]

In [98]:
results['model_address'] = results['model_address'].apply(lambda s : s[20:])

In [99]:
# results = results[results['repo'].apply(lambda x: not ('svg' in x or 'foam' in x))]

In [100]:
import math
results['true_acc'] = (results['acc'] * results['samples']).apply(math.ceil) / results['samples']
results['true_acc'] = results['true_acc'].apply(lambda m: round(m * 100, 2))

In [101]:
results = results[results['split'] != 0.5]
results = results[results['model'] == f't5-{"large" if large else "small"}']

In [102]:
def extract_type_from_address(address):
    for keyword in ['_distance', 'bt', '_length_input', '_length_label', '_conf', 'included', 'ft', 'basic', 'finetuned', 'adapted']:
        if keyword in address:
            if keyword == 'ft' and '0.3_4' in address:
                return 'ft_0.3_4'
            return keyword
    return 'excluded'

In [103]:
results['type'] = results['model_address'].apply(extract_type_from_address)

In [104]:
results['type'].value_counts()

ft               24
adapted          18
finetuned        16
_length_input    16
_distance        16
_length_label    15
excluded          9
included          9
bt                8
Name: type, dtype: int64

In [105]:
# results[results['type'] == '_distance']

In [106]:
f'{1.995:.2f}, {1:.2f}'

'2.00, 1.00'

In [107]:
def print_stats(df):
    print(f"{len(df)}, {(df['true_acc'] * df['samples']).sum() / df['samples'].sum():.2f}, {df['true_acc'].mean():.2f}, {df['true_acc'].median():.2f}")
    print(df[['model_address', 'true_acc', 'samples']])

In [108]:
print_stats(results[(results['type'] == 'included') & (results['repo'] != 'all')])

8, 66.01, 68.39, 63.37
                                        model_address  true_acc  samples
11  ning/t5-large_global_repo-based-included_25-01...     58.26      115
12  ning/t5-large_global_repo-based-included_25-01...     60.78       51
13  ning/t5-large_global_repo-based-included_25-01...     65.96       47
14  ning/t5-large_global_repo-based-included_25-01...     51.22       41
15  ning/t5-large_global_repo-based-included_25-01...    100.00       39
16  ning/t5-large_global_repo-based-included_25-01...     55.00       40
17  ning/t5-large_global_repo-based-included_25-01...     86.49       37
18  ning/t5-large_global_repo-based-included_25-01...     69.44       36


In [109]:
print_stats(results[(results['type'] == 'included') & (results['repo'] == 'all')])

1, 55.01, 55.01, 55.01
                                        model_address  true_acc  samples
21  ning/t5-large_global_repo-based-included_25-01...     55.01     3374


In [110]:
print_stats(results[(results['type'] == 'excluded') & (results['repo'] != 'all')])

8, 54.19, 56.94, 48.58
                                       model_address  true_acc  samples
0  ning/t5-large_global_repo-based_25-01-2022_10-...     46.09      115
1  ning/t5-large_global_repo-based_25-01-2022_10-...     45.10       51
2  ning/t5-large_global_repo-based_25-01-2022_10-...     51.06       47
3  ning/t5-large_global_repo-based_25-01-2022_10-...     39.02       41
4  ning/t5-lar/data/all/data/qooxdoo/qooxdooge_gl...    100.00       39
5  ning/t5-large_global_repo-based_25-01-2022_10-...     40.00       40
6  ning/t5-large_global_repo-based_25-01-2022_10-...     67.57       37
7  ning/t5-large_global_repo-based_25-01-2022_10-...     66.67       36


In [111]:
print_stats(results[(results['type'] == 'excluded') & (results['repo'] == 'all')])

1, 40.01, 40.01, 40.01
                                        model_address  true_acc  samples
10  ning/t5-large_global_repo-based_25-01-2022_10-...     40.01     3374


In [112]:
print_stats(results[(results['type'] == 'finetuned') & (results['repo'] != 'all')])

8, 67.24, 69.94, 64.59
                                        model_address  true_acc  samples
32           finetuned//data/all/data/qooxdoo/qooxdoo     58.26      115
34          finetuned//data/all/data/zloirock/core-js     46.34       41
36          finetuned//data/all/data/emberjs/ember.js     61.70       47
40            finetuned//data/all/data/elastic/kibana     66.67       51
42       finetuned//data/all/data/Encapsule-Annex/onm    100.00       39
44       finetuned//data/all/data/sequelize/sequelize     62.50       40
46              finetuned//data/all/data/dcos/dcos-ui     94.59       37
48  finetuned//data/all/data/LivelyKernel/LivelyKe...     69.44       36


In [113]:
print_stats(results[(results['type'] == 'finetuned') & (results['repo'] == 'all')])

8, 46.39, 46.39, 46.02
                                        model_address  true_acc  samples
33           finetuned//data/all/data/qooxdoo/qooxdoo     45.03     3529
35          finetuned//data/all/data/zloirock/core-js     47.01     3529
37          finetuned//data/all/data/emberjs/ember.js     46.02     3529
41            finetuned//data/all/data/elastic/kibana     47.01     3529
43       finetuned//data/all/data/Encapsule-Annex/onm     46.02     3529
45       finetuned//data/all/data/sequelize/sequelize     46.02     3529
47              finetuned//data/all/data/dcos/dcos-ui     48.00     3529
49  finetuned//data/all/data/LivelyKernel/LivelyKe...     46.02     3529


In [114]:
print_stats(results[(results['type'] == 'adapted') & (results['repo'] != 'all')])

8, 67.49, 69.23, 62.59
      model_address  true_acc  samples
52  /adapted_large/     63.48      115
54  /adapted_large/     58.82       51
56  /adapted_large/     61.70       47
58  /adapted_large/     51.22       41
60  /adapted_large/    100.00       39
62  /adapted_large/     60.00       40
64  /adapted_large/     89.19       37
66  /adapted_large/     69.44       36


In [115]:
print_stats(results[(results['type'] == 'adapted') & (results['repo'] == 'all')])

10, 43.92, 43.92, 44.01
      model_address  true_acc  samples
53  /adapted_large/     43.02     3529
55  /adapted_large/     44.01     3529
57  /adapted_large/     44.01     3529
59  /adapted_large/     45.03     3529
61  /adapted_large/     47.01     3529
63  /adapted_large/     44.01     3529
65  /adapted_large/     45.03     3529
67  /adapted_large/     43.02     3529
69  /adapted_large/     40.01     3529
71  /adapted_large/     44.01     3529


In [116]:
print_stats(results[(results['type'] == 'ft') & (results['repo'] != 'all')])

16, 49.63, 49.66, 51.17
                                        model_address  true_acc  samples
22                         /ft1_986_qooxdoo_0_1/best/     52.17      115
23                          /ft1_825_kibana_0_1/best/     41.18       51
24                        /ft1_177_ember.js_0_1/best/     46.81       47
25                         /ft1_228_core-js_0_1/best/     29.27       41
26                             /ft1_829_onm_0_1/best/     51.28       39
27                       /ft1_717_sequelize_0_1/best/     35.00       40
28                         /ft1_975_dcos-ui_0_1/best/     59.46       37
29                    /ft1_807_LivelyKernel_0_1/best/     58.33       36
72            ftl2_/data/all/data/qooxdoo/qooxdoo_0_1     52.17      115
74             ftl2_/data/all/data/elastic/kibana_0_1     45.10       51
76           ftl2_/data/all/data/emberjs/ember.js_0_1     51.06       47
78           ftl2_/data/all/data/zloirock/core-js_0_1     29.27       41
80        ftl2_/data/all/da

In [117]:
print_stats(results[(results['type'] == 'adapted') & (results['repo'] == 'all')])

10, 43.92, 43.92, 44.01
      model_address  true_acc  samples
53  /adapted_large/     43.02     3529
55  /adapted_large/     44.01     3529
57  /adapted_large/     44.01     3529
59  /adapted_large/     45.03     3529
61  /adapted_large/     47.01     3529
63  /adapted_large/     44.01     3529
65  /adapted_large/     45.03     3529
67  /adapted_large/     43.02     3529
69  /adapted_large/     40.01     3529
71  /adapted_large/     44.01     3529


In [118]:
print_stats(results[(results['type'] == 'adapted') & (results['repo'] != 'all') & ~results['split'].isna()])

8, 67.49, 69.23, 62.59
      model_address  true_acc  samples
52  /adapted_large/     63.48      115
54  /adapted_large/     58.82       51
56  /adapted_large/     61.70       47
58  /adapted_large/     51.22       41
60  /adapted_large/    100.00       39
62  /adapted_large/     60.00       40
64  /adapted_large/     89.19       37
66  /adapted_large/     69.44       36


In [119]:
print_stats(results[(results['type'] == 'ft_0.3_4') & (results['repo'] != 'all')])

0, nan, nan, nan
Empty DataFrame
Columns: [model_address, true_acc, samples]
Index: []


/tmp/ipykernel_20162/898457943.py:2: RuntimeWarning: invalid value encountered in double_scalars
  print(f"{len(df)}, {(df['true_acc'] * df['samples']).sum() / df['samples'].sum():.2f}, {df['true_acc'].mean():.2f}, {df['true_acc'].median():.2f}")


In [120]:
print_stats(results[(results['type'] == 'ft_0.3_4') & (results['repo'] == 'all')])

0, nan, nan, nan
Empty DataFrame
Columns: [model_address, true_acc, samples]
Index: []


/tmp/ipykernel_20162/898457943.py:2: RuntimeWarning: invalid value encountered in double_scalars
  print(f"{len(df)}, {(df['true_acc'] * df['samples']).sum() / df['samples'].sum():.2f}, {df['true_acc'].mean():.2f}, {df['true_acc'].median():.2f}")


In [121]:
print_stats(results[(results['type'] == 'ft') & (results['repo'] == 'all')])

8, 83.91, 83.91, 84.53
                                        model_address  true_acc  samples
73            ftl2_/data/all/data/qooxdoo/qooxdoo_0_1     77.03     1746
75             ftl2_/data/all/data/elastic/kibana_0_1     85.05     1746
77           ftl2_/data/all/data/emberjs/ember.js_0_1     84.02     1746
79           ftl2_/data/all/data/zloirock/core-js_0_1     80.01     1746
81        ftl2_/data/all/data/Encapsule-Annex/onm_0_1     89.00     1746
83        ftl2_/data/all/data/sequelize/sequelize_0_1     85.05     1746
85               ftl2_/data/all/data/dcos/dcos-ui_0_1     88.03     1746
87  ftl2_/data/all/data/LivelyKernel/LivelyKernel_0_1     83.05     1746


In [122]:
print_stats(results[(results['type'] == 'ft') & (results['repo'] != 'all')])

16, 49.63, 49.66, 51.17
                                        model_address  true_acc  samples
22                         /ft1_986_qooxdoo_0_1/best/     52.17      115
23                          /ft1_825_kibana_0_1/best/     41.18       51
24                        /ft1_177_ember.js_0_1/best/     46.81       47
25                         /ft1_228_core-js_0_1/best/     29.27       41
26                             /ft1_829_onm_0_1/best/     51.28       39
27                       /ft1_717_sequelize_0_1/best/     35.00       40
28                         /ft1_975_dcos-ui_0_1/best/     59.46       37
29                    /ft1_807_LivelyKernel_0_1/best/     58.33       36
72            ftl2_/data/all/data/qooxdoo/qooxdoo_0_1     52.17      115
74             ftl2_/data/all/data/elastic/kibana_0_1     45.10       51
76           ftl2_/data/all/data/emberjs/ember.js_0_1     51.06       47
78           ftl2_/data/all/data/zloirock/core-js_0_1     29.27       41
80        ftl2_/data/all/da

In [123]:
print_stats(results[(results['type'] == '_conf') & (results['repo'] != 'all')])

0, nan, nan, nan
Empty DataFrame
Columns: [model_address, true_acc, samples]
Index: []


/tmp/ipykernel_20162/898457943.py:2: RuntimeWarning: invalid value encountered in double_scalars
  print(f"{len(df)}, {(df['true_acc'] * df['samples']).sum() / df['samples'].sum():.2f}, {df['true_acc'].mean():.2f}, {df['true_acc'].median():.2f}")


In [124]:
print_stats(results[(results['type'] == '_conf') & (results['repo'] == 'all')])

0, nan, nan, nan
Empty DataFrame
Columns: [model_address, true_acc, samples]
Index: []


/tmp/ipykernel_20162/898457943.py:2: RuntimeWarning: invalid value encountered in double_scalars
  print(f"{len(df)}, {(df['true_acc'] * df['samples']).sum() / df['samples'].sum():.2f}, {df['true_acc'].mean():.2f}, {df['true_acc'].median():.2f}")


In [125]:
print_stats(results[(results['type'] == '_distance') & (results['repo'] != 'all')])

8, 64.29, 66.60, 62.55
                                         model_address  true_acc  samples
140  currl_/data/all/data/qooxdoo/qooxdoo_distance_...     59.13      115
142  currl_/data/all/data/zloirock/core-js_distance...     46.34       41
144  currl_/data/all/data/emberjs/ember.js_distance...     65.96       47
148  currl_/data/all/data/elastic/kibana_distance_b...     45.10       51
150  currl_/data/all/data/Encapsule-Annex/onm_dista...    100.00       39
152  currl_/data/all/data/sequelize/sequelize_dista...     55.00       40
154   currl_/data/all/data/dcos/dcos-ui_distance_based     94.59       37
156  currl_/data/all/data/LivelyKernel/LivelyKernel...     66.67       36


In [126]:
print_stats(results[(results['type'] == '_distance') & (results['repo'] == 'all')])

8, 46.38, 46.38, 47.50
                                         model_address  true_acc  samples
141  currl_/data/all/data/qooxdoo/qooxdoo_distance_...     41.00     3529
143  currl_/data/all/data/zloirock/core-js_distance...     47.01     3529
145  currl_/data/all/data/emberjs/ember.js_distance...     44.01     3529
149  currl_/data/all/data/elastic/kibana_distance_b...     48.00     3529
151  currl_/data/all/data/Encapsule-Annex/onm_dista...     48.00     3529
153  currl_/data/all/data/sequelize/sequelize_dista...     47.01     3529
155   currl_/data/all/data/dcos/dcos-ui_distance_based     48.00     3529
157  currl_/data/all/data/LivelyKernel/LivelyKernel...     48.00     3529


In [127]:
print_stats(results[(results['type'] == '_length_input') & (results['repo'] != 'all')])

8, 57.14, 60.00, 51.11
                                         model_address  true_acc  samples
91   currl_/data/all/data/qooxdoo/qooxdoo_length_input     48.70      115
93   currl_/data/all/data/zloirock/core-js_length_i...     39.02       41
95   currl_/data/all/data/emberjs/ember.js_length_i...     53.19       47
99    currl_/data/all/data/elastic/kibana_length_input     49.02       51
101  currl_/data/all/data/Encapsule-Annex/onm_lengt...    100.00       39
103  currl_/data/all/data/sequelize/sequelize_lengt...     45.00       40
105     currl_/data/all/data/dcos/dcos-ui_length_input     78.38       37
107  currl_/data/all/data/LivelyKernel/LivelyKernel...     66.67       36


In [128]:
print_stats(results[(results['type'] == '_length_input') & (results['repo'] == 'all')])

8, 47.88, 47.88, 48.00
                                         model_address  true_acc  samples
92   currl_/data/all/data/qooxdoo/qooxdoo_length_input     48.00     3529
94   currl_/data/all/data/zloirock/core-js_length_i...     48.00     3529
96   currl_/data/all/data/emberjs/ember.js_length_i...     48.00     3529
100   currl_/data/all/data/elastic/kibana_length_input     47.01     3529
102  currl_/data/all/data/Encapsule-Annex/onm_lengt...     48.00     3529
104  currl_/data/all/data/sequelize/sequelize_lengt...     48.00     3529
106     currl_/data/all/data/dcos/dcos-ui_length_input     48.00     3529
108  currl_/data/all/data/LivelyKernel/LivelyKernel...     48.00     3529


In [129]:
print_stats(results[(results['type'] == '_length_label') & (results['repo'] == 'all')])

8, 47.88, 47.88, 48.00
                                         model_address  true_acc  samples
111  currl_/data/all/data/qooxdoo/qooxdoo_length_label     47.01     3529
113  currl_/data/all/data/zloirock/core-js_length_l...     48.00     3529
115  currl_/data/all/data/emberjs/ember.js_length_l...     48.00     3529
119   currl_/data/all/data/elastic/kibana_length_label     48.00     3529
121  currl_/data/all/data/Encapsule-Annex/onm_lengt...     48.00     3529
123  currl_/data/all/data/sequelize/sequelize_lengt...     48.00     3529
125     currl_/data/all/data/dcos/dcos-ui_length_label     48.00     3529
127  currl_/data/all/data/LivelyKernel/LivelyKernel...     48.00     3529


In [130]:
print_stats(results[(results['type'] == 'bt') & (results['repo'] != 'all')])

8, 59.85, 63.40, 56.27
                                         model_address  true_acc  samples
130         bt_t5-large//data/all/data/qooxdoo/qooxdoo     48.70      115
131          bt_t5-large//data/all/data/elastic/kibana     47.06       51
132        bt_t5-large//data/all/data/emberjs/ember.js     63.83       47
133        bt_t5-large//data/all/data/zloirock/core-js     46.34       41
134     bt_t5-large//data/all/data/Encapsule-Annex/onm    100.00       39
135     bt_t5-large//data/all/data/sequelize/sequelize     40.00       40
136            bt_t5-large//data/all/data/dcos/dcos-ui     94.59       37
137  bt_t5-large//data/all/data/LivelyKernel/Lively...     66.67       36


In [131]:
print_stats(results[(results['type'] == 'bt') & (results['repo'] == 'all')])

0, nan, nan, nan
Empty DataFrame
Columns: [model_address, true_acc, samples]
Index: []


/tmp/ipykernel_20162/898457943.py:2: RuntimeWarning: invalid value encountered in double_scalars
  print(f"{len(df)}, {(df['true_acc'] * df['samples']).sum() / df['samples'].sum():.2f}, {df['true_acc'].mean():.2f}, {df['true_acc'].median():.2f}")


In [132]:
print_stats(results[(results['type'] == 'bt') & (results['repo'] != 'all')])

8, 59.85, 63.40, 56.27
                                         model_address  true_acc  samples
130         bt_t5-large//data/all/data/qooxdoo/qooxdoo     48.70      115
131          bt_t5-large//data/all/data/elastic/kibana     47.06       51
132        bt_t5-large//data/all/data/emberjs/ember.js     63.83       47
133        bt_t5-large//data/all/data/zloirock/core-js     46.34       41
134     bt_t5-large//data/all/data/Encapsule-Annex/onm    100.00       39
135     bt_t5-large//data/all/data/sequelize/sequelize     40.00       40
136            bt_t5-large//data/all/data/dcos/dcos-ui     94.59       37
137  bt_t5-large//data/all/data/LivelyKernel/Lively...     66.67       36


In [133]:
f"{2.96:.1f}"

'3.0'

In [134]:
results[(results['type'] == 'bt') & (results['repo'] != 'all')].sort_values(by='samples', ascending=False)[['repo', 'true_acc']]

,repo,true_acc
130,/data/all/data/qooxdoo/qooxdoo,48.70
131,/data/all/data/elastic/kibana,47.06
132,/data/all/data/emberjs/ember.js,63.83
133,/data/all/data/zloirock/core-js,46.34
135,/data/all/data/sequelize/sequelize,40.00
134,/data/all/data/Encapsule-Annex/onm,100.00
136,/data/all/data/dcos/dcos-ui,94.59
137,/data/all/data/LivelyKernel/LivelyKernel,66.67


In [135]:
results[(results['type'] == 'by') & (results['repo'] != 'all')].sort_values(by='samples', ascending=False)

,name,repo,acc,#w,samples,time,model,model_address,split,true_acc,type


In [136]:
results[(results['type'] == 'adapted') & (results['repo'] != 'all') & results['split'].isna()]

,name,repo,acc,#w,samples,time,model,model_address,split,true_acc,type


In [137]:
from matplotlib import pyplot as plt

In [138]:
plt.scatter(inc_2['samples'], inc_2['acc'], label='included')
plt.scatter(exc_2['samples'], exc_2['acc'], label='excluded')
plt.scatter(good_2['samples'], good_2['acc'], label='tuned')
plt.scatter(adapter['samples'], adapter['acc'], label='tuned with adapter')
plt.legend()

NameError: name 'inc_2' is not defined

In [ ]:
diff_inc = inc_2.set_index('repo').join(inc_2.set_index('repo')['acc'] - exc_2.set_index('repo')['acc'], rsuffix='ـdiff')

In [ ]:
diff_good = good_2.set_index('repo').join(good_2.set_index('repo')['acc'] - exc_2.set_index('repo')['acc'], rsuffix='ـdiff')

In [ ]:
diff_adapter = adapter.set_index('repo').join(adapter.set_index('repo')['acc'] - exc_2.set_index('repo')['acc'], rsuffix='ـdiff')

In [ ]:
plt.scatter(diff_inc['samples'], diff_inc['acc'], label='included')
plt.scatter(diff_adapter['samples'], diff_adapter['acc'], label='adapter')
plt.legend()

In [ ]:
plt.scatter(diff_good['samples'], diff_good['acc'], label='good')


In [49]:
plt.scatter(diff_adapter['samples'], diff_adapter['acc'], label='adapter')


NameError: name 'diff_adapter' is not defined

In [50]:
import math
results['true_acc'] = (results['acc'] * results['samples']).apply(math.ceil) / results['samples']

In [51]:
results.head(2)

,name,repo,acc,#w,samples,time,model,model_address,split,true_acc,type
0,NaN,/data/all/data/qooxdoo/qooxdoo,0.48,18,115,24-01-2022_16-16-11,t5-small,t5-small_repo-based_21-01-2022_10-29-42/checkp...,0.2,0.486957,excluded
1,NaN,/data/all/data/elastic/kibana,0.43,17,51,24-01-2022_16-17-24,t5-small,t5-small_repo-based_21-01-2022_10-29-42/checkp...,0.2,0.431373,excluded
